In [1]:
import re, sys, copy, numpy as np
from math import sqrt, lcm
from aoclib import list2string
from collections import defaultdict
from itertools import combinations
from operator import add, mul

In [114]:
from collections import deque
import heapq
import time

def reachable_keys(x, y, obtained_keys):
    directions = ((1, 0), (0, 1), (-1, 0), (0, -1))
    out = []
    
    q = deque()
    q.append((x, y, 0, set({(x, y)})))
    while q:
        x, y, d, visited = q.popleft()
        for dir in directions:
            nx, ny = x + dir[0], y + dir[1]
            #print(nx, ny)
            if (nx, ny) in visited:
                continue
            visited.add((nx, ny))
            cell = tunnel[ny][nx]
            if cell == '#' or \
               (cell.isupper() and cell.lower() not in obtained_keys):
                continue
            if cell.islower() and cell not in obtained_keys:
                out.append((nx, ny, d + 1, cell))
                continue
            q.append((nx, ny, d + 1, visited))

    return out

In [116]:
input_tunnel = '#########\n#b.A.@.a#\n#########'
input_tunnel = '########################\n#f.D.E.e.C.b.A.@.a.B.c.#\n######################.#\n#d.....................#\n########################'
input_tunnel = open('18.txt').read().strip()
input_tunnel = open('18input.txt').read().strip()

# Init
available_keys = set()
tunnel = [x.strip() for x in input_tunnel.splitlines()]
for i, row in enumerate(tunnel):
    for j, char in enumerate(row):
            if char == '@':
                current_position = (j, i)
            if char.islower():
                available_keys.add(char)

# BFS algorithm
q = [(0, *current_position, frozenset())]
known = set()
it, t = 0, time.time()
while q:
    if it % 500 == 0:
        print("Duration of 500 iterations:", time.time() - t, "sec")
        t = time.time()
    it += 1
    
    d, x, y, obtained_keys = heapq.heappop(q)

    if obtained_keys == available_keys:
        print("Distance:", d)
        break
        
    if (x, y, obtained_keys) in known:
        continue
    #else:
    #    pass
        #print(d, x, y, "n of obtained:", len(obtained_keys))
        
    known.add((x, y, obtained_keys))
    for (nx, ny, nd, key) in reachable_keys(x, y, obtained_keys):
        heapq.heappush(q, (d + nd, nx, ny, obtained_keys | frozenset([key])))
        #print(nd, key, obtained_keys | frozenset([key]))
        #if it > 100:
            #sys.exit()


Duration of 500 iterations: 0.0002570152282714844 sec
Duration of 500 iterations: 0.3378748893737793 sec
Duration of 500 iterations: 0.37808990478515625 sec
Duration of 500 iterations: 0.42232799530029297 sec
Duration of 500 iterations: 0.31470584869384766 sec
Duration of 500 iterations: 0.1922450065612793 sec
Distance: 4668


In [105]:
from __future__ import print_function, division
import collections
import itertools
import fileinput
import heapq
import sys

# Kind of slow, takes 7s to run :/

grid = [s.rstrip() for s in fileinput.input("18input.txt")]

lingrid = list(itertools.chain.from_iterable(grid))
w, h = len(grid[0]), len(grid)
n = lingrid.index('@')
x, y = n % w, n // w
allkeys = set(c for c in lingrid if c.islower())

#grid[y-1] = grid[y-1][:x]   +  '#'  + grid[y-1][x+1:]
#grid[  y] = grid[y  ][:x-1] + '###' + grid[y  ][x+2:]
#grid[y+1] = grid[y+1][:x]   +  '#'  + grid[y+1][x+1:]

#print('\n'.join(grid))

pos = (
  (x-1, y-1),
  (x+1, y-1),
  (x-1, y+1),
  (x+1, y+1),
)

pos = (
    (x, y),
)


def reachable_keys_(sx, sy, keys):
    q = collections.deque([(sx, sy, 0)])
    #print(q)
    seen = set()
    d = ( (-1, 0), (1, 0), (0, -1), (0, 1) )
    while q:
        cx, cy, l = q.popleft()
        if grid[cy][cx].islower() and grid[cy][cx] not in keys:
            yield l, cx, cy, grid[cy][cx]
            continue
        for dx, dy in d:
            nx, ny = cx + dx, cy + dy
            if ((nx, ny)) in seen:
                continue
            seen.add((nx, ny))

            c = grid[ny][nx]
            if c != '#' and (not c.isupper() or c.lower() in keys):
                q.append((nx, ny, l + 1))


# Shortest path from node "no keys" to node "all keys".
q = [(0, pos, frozenset())]
seen = set()

it = 0
t = time.time()
dlim = 1000
while q:
    if it % 500 == 0:
        print("Duration of 500 iterations:", time.time() - t)
    it += 1
    
    #print(q)
    #print(q[0])
    d, cpos, keys = heapq.heappop(q)
    #print(cpos)
    #print(sorted(keys))
    if keys == allkeys:
        print(d)
        break

    if (cpos, keys) in seen:
        continue
    #else:
    #    print(d, len(keys))
    
    seen.add((cpos, keys))

    for i, (cx, cy) in enumerate(cpos):
        for l, nx, ny, key in reachable_keys_(cx, cy, keys):
            print(l, key, keys | frozenset([key]))
            npos = cpos[0:i] + ((nx, ny),) + cpos[i+1:]
            heapq.heappush(q, (d + l, npos, keys | frozenset([key])))
            if it > 100:
                sys.exit()


Duration of 500 iterations: 0.00023412704467773438
16 w frozenset({'w'})
72 i frozenset({'i'})
12 j frozenset({'w', 'j'})
86 i frozenset({'w', 'i'})
18 q frozenset({'q', 'w', 'j'})
34 d frozenset({'d', 'w', 'j'})
98 i frozenset({'w', 'i', 'j'})
20 d frozenset({'d', 'q', 'w', 'j'})
116 i frozenset({'q', 'w', 'i', 'j'})
20 q frozenset({'d', 'q', 'w', 'j'})
74 s frozenset({'d', 's', 'w', 'j'})
132 i frozenset({'d', 'w', 'i', 'j'})
74 s frozenset({'s', 'j', 'd', 'q', 'w'})
132 i frozenset({'j', 'd', 'q', 'i', 'w'})
164 t frozenset({'j', 't', 'd', 'q', 'w'})
180 u frozenset({'j', 'u', 'd', 'q', 'w'})
86 w frozenset({'w', 'i'})
94 s frozenset({'s', 'j', 'd', 'q', 'w'})
116 i frozenset({'j', 'd', 'q', 'i', 'w'})
148 t frozenset({'j', 't', 'd', 'q', 'w'})
164 u frozenset({'j', 'u', 'd', 'q', 'w'})
98 j frozenset({'w', 'i', 'j'})
160 v frozenset({'v', 'w', 'i'})
116 q frozenset({'q', 'w', 'i', 'j'})
132 d frozenset({'d', 'w', 'i', 'j'})
160 v frozenset({'v', 'w', 'i', 'j'})
94 q frozenset({'s',

SystemExit: 